## Encontrando conectores chaves

Temos uma lista de pessoas com seus respectivos "ids". 
Logo abaixo mostramos quem se relaciona com quem, através do objeto "friendships". Ex: O usuário 0 ou Hero, se relaciona com o usuário 1 ou Dunn e usuário 2 ou Sue, e assim por diante.

In [1]:
users = [
    {"id": 0, "name": "Hero"},
    {"id": 1, "name": "Dunn"},
    {"id": 2, "name": "Sue"},
    {"id": 3, "name": "Chi"},
    {"id": 4, "name": "Thor"},
    {"id": 5, "name": "Chive"},
    {"id": 6, "name": "Hicks"},
    {"id": 7, "name": "Devin"},
    {"id": 8, "name": "Kate"},
    {"id": 9, "name": "Klein"}    
]
print (users)

[{'id': 0, 'name': 'Hero'}, {'id': 1, 'name': 'Dunn'}, {'id': 2, 'name': 'Sue'}, {'id': 3, 'name': 'Chi'}, {'id': 4, 'name': 'Thor'}, {'id': 5, 'name': 'Chive'}, {'id': 6, 'name': 'Hicks'}, {'id': 7, 'name': 'Devin'}, {'id': 8, 'name': 'Kate'}, {'id': 9, 'name': 'Klein'}]


In [2]:
friendships = [(0,1), (0,2), (1,2), (1,3), (2,3), (3,4), (4,5), (5,6), (5,7), (6,8), 
              (7,8), (8,9)]

Agora queremos adicionar uma lista de amigos para cada usuário no objeto "users"

In [3]:
for user in users:
    user["friends"] = []

In [4]:
for i, j in friendships:
    #isso funciona porque user[i] é o usuário cuja id é i 
    users[i]["friends"].append(users[j]) #adiciona i como amigo de j 
    users[j]["friends"].append(users[i]) #adiciona j como amigo de i 

A partir da lista de amizades criadas para casa usuário queremos definir quantas conexões (relacionamentos) cada usuário possui:

In [5]:
def number_of_friends(user):
    """quantos amigos o usuário tem?"""
    return len(user["friends"])          #tamanho da lista friends_ids

total_connections = sum(number_of_friends(user)
                      for user in users)

Em alguns casos é interessante calcular o número médio de conexões que da rede como um todo:

In [6]:
from __future__ import division                 #divisao inteira esta incompleta
num_users = len(users)                          #tamanho da lista de usuarios
avg_connections = total_connections / num_users #2.4

Abaixo temos uma lista que corresponde ao número do id do usuário e quantas conexões ele possui.

In [7]:
#cria uma lista (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user))
                    for user in users]

print(num_friends_by_id)

[(0, 2), (1, 3), (2, 3), (3, 3), (4, 2), (5, 3), (6, 2), (7, 2), (8, 3), (9, 1)]


In [8]:
###DEU ERRO, MAS É SÓ PARA ORDENAR OS USUARIOS DE ACORDO COM SEU NUMERO DE CONEXÕES
sorted(num_friends_by_id,                               #é ordenado
      key=lambda(user_id, num_of_friends): num_of_friends,    #por num_friends
      reverse = True)                                   #do maior pro menor

#cada par é (user_id, num_friends)
#[(1,3), (2,3), (3,3), (5,3), (8,3),
#  (0,2), (4,2), (6,2), (7,2), (9,1)]

SyntaxError: invalid syntax (<ipython-input-8-904cfd831c8b>, line 3)

Para descobrir quem são os amigos de cada um dos usuários, utilizamos:

In [9]:
def friends_of_friends_ids_bad(user):
    #'foaf' é abreviação de 'friends of a friend'
    return [foaf["id"]
           for friend in user ["friends"]         #Para cada amigo de usuário
           for foaf in friend ["friends"]]        #Para cada _their_friends

friends_of_friends_ids_bad(users[0])

[0, 2, 3, 0, 1, 3]

Para o usuário 0 ou "Hero" os amigos dos seus amigos são os usuários dos ids 0,2,3,0,1,3. 
Ele mesmo aparece nessa lista porque ele é amigo dos seus amigos. 
Perceba que a função nos retorna todas as pessoas com que os amigos de Hero tem conexão, inclusive ele. 

In [10]:
print ([friend["id"] for friend in users[0]["friends"]])
print ([friend["id"] for friend in users[1]["friends"]])
print ([friend["id"] for friend in users[2]["friends"]])


[1, 2]
[0, 2, 3]
[0, 1, 3]


Para avaliarmos a quantidade de amigos em comum que um usuário tem com os demais, fazemos a seguinte função:

In [11]:
from collections import Counter

def not_the_same (user, other_user):
    """ dois usuários não são os mesmos se possuem ids diferentes"""
    return user["id"] != other_user["id"]

def not_friends (user, other_user):
    """"other_user não é um amigo se não está em user["friends"];
    isso é, se é not_the_same com todas as pessoas em user["friends"]"""
    return all(not_the_same(friend, other_user)
              for friend in user["friends"])

def friends_of_friend_ids(user):
    return Counter (foaf ["id"]
                   for friend in user["friends"]     #para cada um dos meus amigos
                   for foaf in friend["friends"]     #que contam *their* amigos 
                   if not_the_same(user, foaf)       #que não sejam eu 
                   and not_friends(user, foaf))      #e que não sejam meus amigos

print (friends_of_friend_ids(users[3]))

Counter({0: 2, 5: 1})


Também podemos conectar pessoas através dos seus interesses em comum. No objeto a seguir temos uma lista, que contém os ids dos usuários e seus respectivos interesses:

In [12]:
interests = [
    (0, "Hadoop"), (0, "Big Data"), (0,"HBase"), (0, "Java"),
    (0, "Spark"), (0, "Storm"), (0, "Cassandra"),
    (1, "NoSQL"), (1, "MongoDB"), (1, "Cassandra"), (1, "HBase"), 
    (1, "Postgres"), (2, "Phyton"), (2, "scikit-learn"), (2, "scipy"), 
    (2, "numpy"), (2, "statsmodels"), (2, "pandas"), (3, "R"), (3, "Phyton"), 
    (3, "statistics"), (3, "regression"), (3, "probability"), 
    (4, "machine learning"), (4, "regression"), (4, "decision trees"), 
    (4, "libsvm"), (5, "Phyton"), (5, "R"), (5, "Java"), (5, "C++"), 
    (5, "Haskell"), (5, "programming languages"), (6, "statistics"), 
    (6, "probability"), (6, "mathematics"), (6, "theory"), 
    (7, "machine learning"), (7, "scikit-learn"), (7, "Mahout"), 
    (7, "neural networks"), (8, "neural networks"), (8, "deep learning"), 
    (8, "Big Data"), (8, "artificial intelligence"), (9, "Hadoop"), 
    (9, "Java"), (9, "MapReduce"), (9, "Big Data")
]

Para criar uma função para conectarmos os usuários com o mesmo interesse, temos:

In [13]:
def data_scientists_who_like(target_interest):
    return [user_id
           for user_id, user_interest in interests
           if user_interest == target_interest]

In [14]:
data_scientists_who_like ("R")

[3, 5]

Com essa função, colocamos o interesse desejado e ela nos retorna quais usuários se interessam por esse assunto. Porém, ela não é muito prática quando temos muitos usuários, ou muitos interesses pois avaliar um por um daria muito trabalho. 
Assim, podemos criar um índice de interesses para usuários

In [15]:
from collections import defaultdict 

#as chaves são interesses, os valores são listas de users_ids com interesses 
user_ids_by_interest = defaultdict(list)

for user_id, interest in interests:
    user_ids_by_interest[interest].append(user_id)
    


In [16]:
E outro de usuários para interesses:

SyntaxError: invalid syntax (<ipython-input-16-41aabc195b90>, line 1)

In [17]:
#as chaves são user_ids, os valores são as listas de interests para aquele user_id
interests_by_user_id = defaultdict(list)

for user_id, interest in interests: 
    interests_by_user_id[user_id].append(interest)

Agora fica fácil descobrir quem possui os maiores interesses em comum com um certo usuário:

* Itera sobre os interesses do usuário
* Para cada interesse, itera sobre os outros usuários com aquele interesse. 
* Mantém a contagem de quantas vezes vemos cada outro usuário

In [18]:
def most_commom_interests_with(user):
    return Counter(interested_user_id
        for interest in interests_by_user_id[users["id"]]
        for interested_user_id in user_ids_by_interest[interest]
        if interested_user_id != user["id"])